## Installs and Imports

In [2]:
!pip install open3d

In [2]:
import numpy as np
import open3d as o3d

## Load Files

In [3]:
print("Load a ply point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud("plyfolder/points0.ply")
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

Load a ply point cloud, print it, and render it
PointCloud with 24775 points.
[[-8.684366 -8.684366 44.307988]
 [-8.567612 -8.676063 44.265625]
 [-8.461217 -8.678172 44.276386]
 ...
 [ 9.860027 10.116131 52.266242]
 [10.000024 10.128229 52.328747]
 [10.131802 10.131802 52.347209]]


## Outlier Removal

In [22]:
downpcd = pcd.voxel_down_sample(voxel_size=0.1)

In [23]:
o3d.visualization.draw_geometries([downpcd])

## Mesh Convertion

In [26]:
alpha = 0.7

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(downpcd, alpha)
mesh.compute_vertex_normals()
print(mesh)

TriangleMesh with 21367 points and 51648 triangles.


In [28]:
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)